## Libraries

In [4]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
import math
import matplotlib.image as mpimg
%matplotlib inline

## Step 0.0 : get home locations as for the replication part

In [5]:
DATA_FOLDER = 'data/'

B_checkins = 'loc-brightkite_totalCheckins.txt.gz'
G_checkins = 'loc-gowalla_totalCheckins.txt.gz'

brightkite_checkins = pd.read_table(DATA_FOLDER + B_checkins, usecols = [0, 1, 2, 3], names = ['user', 'time', 'lat', 'lon'], compression = 'gzip')
gowalla_checkins = pd.read_table(DATA_FOLDER + G_checkins, usecols = [0, 1, 2, 3], names = ['user', 'time', 'lat', 'lon'], compression = 'gzip')

In [ ]:
#preprocessing data to remove weird or null points
null_points = brightkite_checkins[(brightkite_checkins['lat'] == 0) & (brightkite_checkins['lon'] == 0)]
brightkite_checkins.drop(null_points.index, inplace = True)
null_points = gowalla_checkins[(gowalla_checkins['lat'] == 0) & (gowalla_checkins['lon'] == 0)]
gowalla_checkins.drop(null_points.index, inplace = True)
outliers_lat_B = brightkite_checkins[(brightkite_checkins['lat'] < -90) | (brightkite_checkins['lat'] > 90)]
brightkite_checkins.drop(outliers_lat_B.index, inplace = True)
outliers_lat_G = gowalla_checkins[gowalla_checkins['lat'] > 90]
gowalla_checkins.drop(outliers_lat_G.index, inplace = True)
brightkite_checkins.dropna(inplace = True)
gowalla_checkins.dropna(inplace = True)

In [ ]:
#one of your code, to get brightkite_home_location and gowalla_home_location

In [3]:
#brightkite_home_location.to_csv('intermediaryDataframe/home_loc_B.csv', sep = ';', index_col = 'user')
#gowalla_home_locations.to_csv('intermediaryDataframe/home_loc_G.csv', sep = ';', index_col = 'user')
brightkite_home_location = pd.read_csv('intermediaryDataframe/home_loc_B.csv', sep = ';', index_col = 'user')
gowalla_home_locations = pd.read_csv('intermediaryDataframe/home_loc_G.csv', sep = ';', index_col = 'user')

NameError: name 'pd' is not defined

## Step 0.1 : associate a country to each home location and check-in

In [1]:
import reverse_geocoder as rg

In [4]:
L_G = []                              #list of home locations coordinates
for i in range(len(gowalla_home_locations)):
    L_G += [(gowalla_home_locations.iat[i, 0], gowalla_home_locations.iat[i, 1])]
results_G = rg.search(L_G)
gowalla_home_locations['country'] = [results_G[i].get('cc') for i in range(len(gowalla_home_locations))]
gowalla_home_locations.sample(10)

Loading formatted geocoded file...


,lat,lon,country
user,,,
91835,37.342151,-121.955892,US
102478,34.235469,108.899579,CN
139247,61.147470,10.455413,NO
107210,57.780983,14.181546,SE
183075,52.369274,4.899398,NL
104379,21.520912,39.209301,SA
195040,57.700524,11.962637,SE
43387,37.501109,127.025271,KR
14751,49.455817,11.082283,DE


In [ ]:
#same for Brightkite
L_B = []
for i in range(len(brightkite_home_location)):
    L_B += [(brightkite_home_location.iat[i, 0], brightkite_home_location.iat[i, 1])]
results_B = rg.search(L_B)
brightkite_home_location['country'] = [results_B[i].get('cc') for i in range(len(home_loc_B))]

Now determine check-ins countries, using the exact same method

In [ ]:
L_G_checkins = []
for i in range(len(gowalla_checkins)):
    L_G_checkins += [(gowalla_checkins.iat[i, 2], gowalla_checkins.iat[i, 3])]
results_G_checkins = rg.search(L_G_checkins)
gowalla_checkins['country'] = [results_G_checkins[i].get('cc') for i in range(len(gowalla_checkins))]

In [ ]:
#same for Brightkite
L_B_checkins = []
for i in range(len(brightkite_checkins)):
    L_B_checkins += [(brightkite_checkins.iat[i, 2], brightkite_checkins.iat[i, 3])]
results_B_checkins = rg.search(L_B_checkins)
brightkite_checkins['country'] = [results_B_checkins[i].get('cc') for i in range(len(brightkite_checkins))]

## Additional data

In [ ]:
POPULATION_DATASET = DATA_FOLDER+"pop_per_country.csv"
COUNTRY_CODES = DATA_FOLDER+"countrycodes.csv"

population = pd.read_csv(POPULATION_DATASET, index_col = 0, names = ['country', 'population'], header = 0)
country_codes = pd.read_csv(COUNTRY_CODES, sep =',')